In [12]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
dummy_data = pd.read_csv('/Users/henry/Desktop/dummdata - Sheet1.csv')

#Buckets + outlier detection and remove/replacement

#Class for bucketing Data
#Class for outlier detection/replcaement or removal
#Class for one hot encoding
#Class for scaling numerical values

In [192]:
#either min or max bucket has to be inclusive add option. 
class Even_Bucketing_Sizes:
    
    def __init__(self, raw_data):
        self.raw_data = raw_data
        if isinstance(raw_data, pd.DataFrame):
            pass
        else:
            raise ValueError('Must pass in a dataframe')
    
   
    def bucketing(self, bucket_type='even_compartments', columns=[], Number_of_buckets = 10, bucket_array =[], replace = False, append=True,
                 min_bucket_size = None, max_bucket_size = None, bottom_bucket_inclusive =True):
        if bucket_type == 'even_entries':
            self.even_entry_buckets(columns, Number_of_buckets)
        elif bucket_type == 'even_bucket_sizes':
            data_T = self.even_size_buckets(columns, Number_of_buckets,replace,append, min_bucket_size,max_bucket_size, bottom_bucket_inclusive)
            return data_T
        elif bucket_type == 'custom':
            data_T = self.custom_buckets(columns, bucket_array, replace, append, bottom_bucket_inclusive)
            return data_T
        elif bucket_type == 'rate_of_change':
            self.minmax_bs()
        else:
            raise ValueError('invalid bucket type options are: custom, even_entries, even_bucket_sizes and min_max_bs')
            
    #Gives buckets with an around an even number of entries per bucket with given number of buckets.
    def even_entry_buckets(self,columns = []):
        print('even_entries_per_bucket_buckets')
        
    #Gives even bucket sizes
    def even_size_buckets(self,columns, Number_of_buckets,replace,append, min_bucket_size,max_bucket_size, bottom_bucket_inclusive):
        if columns[0]:
            pass
        else:
            raise ValueError('columns must be a list of columns')
        if str(Number_of_buckets).isnumeric():
            pass
        else:
            raise ValueError('Number of buckets must be a number')
        if replace == True or replace == False:
            pass
        else:
            raise ValueError('replace must be boolean')
        if append == True or append == False:
            pass
        else:
            raise ValueError('append must be boolean')
        
        print('even bucket sizes: ', columns)
        return_frame = pd.DataFrame()
        if bottom_bucket_inclusive == True: 
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                if str(min_bucket_size).isnumeric():
                    minimum = min_bucket_size
                else:
                    minimum = min(data)
                if str(max_bucket_size).isnumeric():
                    maximum = max_bucket_size
                else:
                    maximum = max(data)
                try:
                    bucket_size = (maximum-minimum)/Number_of_buckets
                    print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                bucket_start = minimum
                

                for bucket in range(0, int(Number_of_buckets)):
                    bucket_end = bucket_start + bucket_size
                    if bucket == 0:
                        less_than_min_column_name = str(item) + '_<_' + str(minimum)
                        bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] < minimum else 0,axis=1)


                    column_name = str(item)+ '_>=_' + str(bucket_start) + '_<' + str(bucket_end) 
                    bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= bucket_start and x[item] < bucket_end else 0,axis=1)
                    if bucket == int(Number_of_buckets) -1:
                        more_than_max_column_name = str(item) + '_>=_' + str(maximum)
                        bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]>= maximum else 0,axis=1)

                    bucket_start +=bucket_size
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
        else:
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                if str(min_bucket_size).isnumeric():
                    minimum = min_bucket_size
                else:
                    minimum = min(data)
                if str(max_bucket_size).isnumeric():
                    maximum = max_bucket_size
                else:
                    maximum = max(data)
                try:
                    bucket_size = (maximum-minimum)/Number_of_buckets
                    print('\ncolumn: ', item, '\nmin:', str(minimum), ' max:', str(maximum), ' bucket size:', str(bucket_size))
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                bucket_start = minimum
                
                for bucket in range(0, int(Number_of_buckets)):
                    bucket_end = bucket_start + bucket_size
                    if bucket == 0:
                        less_than_min_column_name = str(item) + '_<=_' + str(minimum)
                        bucketed_data[less_than_min_column_name] = bucketed_data.apply(lambda x: 1 if x[item] <= minimum else 0,axis=1)


                    column_name = str(item)+ '_>_' + str(bucket_start) + '_<=' + str(bucket_end) 
                    bucketed_data[column_name] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start and x[item] <= bucket_end else 0,axis=1)
                    if bucket == int(Number_of_buckets) -1:
                        more_than_max_column_name = str(item) + '_>_' + str(maximum)
                        bucketed_data[more_than_max_column_name] = bucketed_data.apply(lambda x: 1 if x[item]> maximum else 0,axis=1)

                    bucket_start +=bucket_size
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
                
                
        if append == True and replace==True:
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==True and replace==False:
            return_frame = return_frame.drop(columns=columns)
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return return_frame
        elif append==False and replace==True:
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==False and replace==False:
            return return_frame
       
    
    
    
    
    def custom_buckets(self, columns,bucket_array, replace, append, bottom_bucket_inclusive):
        bucket_array.sort()
        try:
            columns[0]
            bucket_array[0]
        except:
            raise ValueError('columns must be a list of columns and bucket_array must be an array of bottom bucket sizes')
        if replace == True or replace == False:
            pass
        else:
            raise ValueError('replace must be boolean')
        if append == True or append == False:
            pass
        else:
            raise ValueError('append must be boolean')
        
        print('custom bucket sizes: ', columns)
        return_frame = pd.DataFrame()
        if bottom_bucket_inclusive == True: 
            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)
                
                try:
                   
                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')
                    
                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>='+ str(bucket_start)+ '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end and x[item] >= bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] < bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>=' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] >= bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)
                
                
            if append == True and replace==True:
                return_frame = pd.concat([self.raw_data, return_frame],axis=1)
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==True and replace==False:
                return_frame = return_frame.drop(columns=columns)
                return_frame = pd.concat([self.raw_data, return_frame],axis=1)
                return return_frame
            elif append==False and replace==True:
                return_frame = return_frame.drop(columns=columns)
                return return_frame
            elif append==False and replace==False:
                return return_frame
       
        else: 

            for item in columns:
                try: 
                    bucketed_data = pd.DataFrame(self.raw_data[item], columns=[item])
                except:
                    raise ValueError('no such column: ', item)

                try:

                    print('\ncolumn: ', item, 'columns:', str(bucket_array))
                    test = float(bucketed_data[item].iloc[0])
                except:
                    raise ValueError('Bucket column ', item, ' must be numeric')

                for bucket in range(0,len(bucket_array)+1):
                    try:
                        if bucket != 0 and bucket != (len(bucket_array)):
                            bucket_start = bucket_array[bucket-1]
                            bucket_end = bucket_array[bucket]
                            new_column = item +'_>'+ str(bucket_start)+ '_<=' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end and x[item] > bucket_start else 0,axis=1)
                        elif bucket == 0 and bucket != len(bucket_array) + 1:
                            bucket_end = bucket_array[bucket]
                            new_column = item + '_<=' + str(bucket_end)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] <= bucket_end else 0,axis=1)
                        else:
                            bucket_start = bucket_array[bucket-1]
                            new_column = item + '_>' + str(bucket_start)
                            bucketed_data[new_column] = bucketed_data.apply(lambda x: 1 if x[item] > bucket_start else 0,axis=1)
                    except:
                        raise ValueError('Buckets must be an array of bottom bucket values')
                return_frame = pd.concat([return_frame, bucketed_data], axis=1)


        if append == True and replace==True:
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==True and replace==False:
            return_frame = return_frame.drop(columns=columns)
            return_frame = pd.concat([self.raw_data, return_frame],axis=1)
            return return_frame
        elif append==False and replace==True:
            return_frame = return_frame.drop(columns=columns)
            return return_frame
        elif append==False and replace==False:
            return return_frame


                    
       
                
            
        
    
        
        
        

In [254]:
#Min and Max will be even entries between those values will get as close as possible to even entries
#How to do when two bucket bottoms are the same and how to deal with first and last buckets. 

#Need to return alot of the stuff such as buckets for this one to the user so can use in scoring. 
def even_entry_buckets(data,columns, Number_of_buckets, replace, append, bottom_bucket_inclusive):
        bucket_df = pd.DataFrame()
        run_through = 0
        for column in columns:
            entries = list(dummy_data[column])
            entries.sort()
            items_per_bucket = len(entries)/Number_of_buckets
            bucket_iter = 0 
            buckets = []
            for i in range(0, Number_of_buckets):
                bucket_iter+=int(items_per_bucket)
                bucket = entries[bucket_iter]
                buckets.append(bucket)
                
        
            
            just_the_data = Bucketing_Data(raw_data=data).bucketing(bucket_type='custom', columns = [column], bucket_array=buckets, replace=True, append=False, bottom_bucket_inclusive=bottom_bucket_inclusive)
            if run_through == 0:
                bucket_df = just_the_data
            else:
                bucket_df = bucket_df.join(just_the_data)
            run_through +=1
        return bucket_df

data = even_entry_buckets(data = dummy_data, columns=['temperature', 'windspeed'], Number_of_buckets=10, replace=True, append=True, bottom_bucket_inclusive=False)
            
    
    
    
for column in data.columns:
    print(column, data[column].sum())

custom bucket sizes:  ['temperature']

column:  temperature columns: [10, 19, 31, 41, 51, 61, 71, 79, 90, 99]
custom bucket sizes:  ['windspeed']

column:  windspeed columns: [13, 16, 19, 22, 25, 28, 31, 34, 37, 40]
temperature_<=10 50
temperature_>10_<=19 54
temperature_>19_<=31 51
temperature_>31_<=41 43
temperature_>41_<=51 52
temperature_>51_<=61 46
temperature_>61_<=71 50
temperature_>71_<=79 48
temperature_>79_<=90 53
temperature_>90_<=99 45
temperature_>99 7
windspeed_<=13 63
windspeed_>13_<=16 39
windspeed_>16_<=19 50
windspeed_>19_<=22 61
windspeed_>22_<=25 42
windspeed_>25_<=28 40
windspeed_>28_<=31 49
windspeed_>31_<=34 55
windspeed_>34_<=37 43
windspeed_>37_<=40 57
windspeed_>40 0


In [206]:
# #Even bucket sizes. 
# my_data = Bucketing_Data(raw_data=dummy_data).bucketing(bucket_type='even_bucket_sizes', columns=['temperature'],Number_of_buckets=5, replace=False,append=True, min_bucket_size=30, max_bucket_size=70, bottom_bucket_inclusive=False)
# my_data.head(50)

#Custom Buckets
#object_ = Bucketing_Data(raw_data=dummy_data)
# test = object_.bucketing(bucket_type='custom', columns=['temperature', 'windspeed'], 
#                   bucket_array=[10,20,40,30,70.5,100,110],replace=False, append=False, bottom_bucket_inclusive=False)
                 

           

